# **IMPORTING AUDIO**

In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

## **Model Parameters for Speech Recognition or Language Processing**

In [ ]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


# **Setup Environment for Speech Recognition and Speaker Diarization**

In [ ]:
import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
# Use 'cpu' if no GPU is available or preferred
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cpu")) # Changed device to 'cpu'

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

# **CONVERTING AUDIO TO WAV**

In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

# **VARIABLE FOR MODEL**

In [ ]:
model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 2.88G/2.88G [00:18<00:00, 169MiB/s]


# **Transcribe Audio and Extract Segments Using Whisper Model**

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

# **Calculate Duration of Audio File Using wave Module**

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

# **Extract Segment Embedding from Audio Using Pretrained Model**

In [ ]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [ ]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  # Ensure waveform is single-channel
  waveform = waveform[:, 0, :]  # Select the first channel if there are multiple
  return embedding_model(waveform[None])

In [ ]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  # Handle the case where waveform might be single-channel
  if waveform.ndim == 3:  # Check if waveform has 3 dimensions
      waveform = waveform[:, 0, :]  # Select the first channel if there are multiple
  return embedding_model(waveform[None])

# **Generate and Write Formatted Transcript from Segmented Transcription Data**

In [ ]:
import datetime

def time(secs):
    return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for i, segment in enumerate(segments):
    # Assign speakers based on the index: even indices for "Customer Care," odd for "Client"
    if i % 2 == 0:
        speaker = "Customer Care"
    else:
        speaker = "Client"

    start_time = time(segment["start"])

    # Write the speaker and start time if it's the first segment or if the speaker has changed
    if i == 0 or (i > 0 and (i - 1) % 2 != i % 2):
        f.write(f"\n{speaker} {start_time}\n")

    # Write the text of the segment if it exists, ensuring each segment is on a new line
    if "text" in segment:
        f.write(segment["text"].strip() + '\n')

f.close()


In [ ]:
print(open("transcript.txt","r").read())


Customer Care 0:00:00
Hello, thank you for calling Healthcare Solutions. This is Jain Do speaking. How can I assist you today?

Client 0:00:08
Hi Jain, this is John Smith. I have a question about my recent bill from the New York branch.

Customer Care 0:00:13
Sure John, I can help with that. I see that you were charged $200 on June 20, 2023.

Client 0:00:20
Let me check if there are any available discounts or promotions for you.

Customer Care 0:00:24
That would be great. Also, I recently moved to the 123 Elam Street in Los Angeles.

Client 0:00:28
Thank you for the update. I see here that we have a special promotion for new residents in California.

Customer Care 0:00:35
You are eligible for a 10% discount on your next bill.

Client 0:00:38
Thank you Jain. That's very helpful.

Customer Care 0:00:41
You are welcome John.



# **HIPPA RULES**

In [ ]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

# Read the transcript
with open("transcript.txt", "r") as file:
    transcript = file.read()

# Process the transcript with SpaCy
doc = nlp(transcript)

# Define the PHI entities to look for
phi_entities = ["PERSON", "GPE", "DATE", "ORG", "LOC", "NORP", "FAC", "EVENT"]

# Dictionary to keep track of detected entities
detected_entities = {entity: False for entity in phi_entities}

# Check for PHI in the transcript
for entity in doc.ents:
    if entity.label_ in detected_entities:
        detected_entities[entity.label_] = True
        print(f"Detected: {entity.text} ({entity.label_})")

# Check if all entities are detected
all_detected = all(detected_entities.values())

if all_detected:
    print("All required entities are present in the transcript.")
else:
    print("NOT ASKED: The call does not contain all required entity types.")
    for entity, detected in detected_entities.items():
        if not detected:
            print(f"Missing entity type: {entity}")



Detected: Customer Care 0:00:00 (ORG)
Detected: Healthcare Solutions (ORG)
Detected: Jain (PERSON)
Detected: today (DATE)
Detected: John Smith (PERSON)
Detected: New York (GPE)
Detected: 0:00:13 (DATE)
Detected: Sure John (PERSON)
Detected: June 20, 2023 (DATE)
Detected: the 123 Elam Street (ORG)
Detected: Los Angeles (GPE)
Detected: California (GPE)
Detected: 0:00:38 (DATE)
Detected: Jain (PERSON)
Detected: John (PERSON)
NOT ASKED: The call does not contain all required entity types.
Missing entity type: LOC
Missing entity type: NORP
Missing entity type: FAC
Missing entity type: EVENT


# **DISCOUNT AND OFFER**

In [ ]:
import re

# Define a list of keywords and phrases that indicate a discount or offer
discount_keywords = [
    "discount", "offer", "promotion", "deal", "special price",
    "reduced rate", "sale", "saving", "price cut", "coupon", "rebate", "markdown"
]

# Read the transcript
with open("transcript.txt", "r") as file:
    transcript = file.read()

# Convert transcript to lower case for case-insensitive matching
transcript_lower = transcript.lower()

# Search for discount keywords and discount percentage in the transcript
discount_found = False
discount_percentage = None

for keyword in discount_keywords:
    keyword_pattern = r'\b' + re.escape(keyword) + r'\b'
    if re.search(keyword_pattern, transcript_lower):
        discount_found = True
        # Look for percentage values near the keyword
        match = re.search(r'(\d+)%', transcript_lower)
        if match:
            discount_percentage = match.group(1)
        print(f"Discount/Offer Detected: '{keyword}' in the transcript.")
        break

if discount_found:
    if discount_percentage:
        print(f"Discount Percentage Detected: {discount_percentage}%")
    else:
        print("No specific discount percentage mentioned.")
else:
    print("No discount or offer detected in the transcript.")

Discount/Offer Detected: 'discount' in the transcript.
Discount Percentage Detected: 10%


# **Summary**

In [ ]:
!pip install transformers

from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Read the transcript
with open("transcript.txt", "r") as file:
    transcript = file.read()

# Generate a summary
summary = summarizer(transcript, max_length=150, min_length=30, do_sample=False)[0]['summary_text']

# Print the summary
print("Summary of the conversation:")
print(summary)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Summary of the conversation:
John Smith calls to ask about his recent bill from the New York branch. John recently moved to the 123 Elam Street in Los Angeles. John is eligible for a 10% discount on his next bill.


# **SENTIMENT ANALYZER**

In [ ]:
# Load the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Read the transcript
with open("transcript.txt", "r") as file:
    transcript = file.read()

# Analyze the sentiment of the entire conversation
conversation_sentiment = sentiment_analyzer(transcript)

# Function to convert sentiment score to percentage
def score_to_percentage(score):
    return {
        "NEGATIVE": "Negative",
        "NEUTRAL": "Neutral",
        "POSITIVE": "Positive"
    }[score]

# Convert sentiment score to percentage
converted_sentiment = {
    "label": score_to_percentage(conversation_sentiment[0]['label']),
    "score": round(conversation_sentiment[0]['score'] * 100, 2)
}

# Print sentiment of the entire conversation in percentage form
print("Sentiment of the entire conversation:")
print(converted_sentiment)

# Optional: Analyze the sentiment of each segment
with open("transcript.txt", "r") as file:
    lines = file.readlines()

segment_sentiments = []
for line in lines:
    if line.strip():  # Skip empty lines
        segment_sentiment = sentiment_analyzer(line.strip())
        converted_segment_sentiment = {
            "label": score_to_percentage(segment_sentiment[0]['label']),
            "score": round(segment_sentiment[0]['score'] * 100, 2)
        }
        segment_sentiments.append((line.strip(), converted_segment_sentiment))

print("\nSentiment of each segment:")
for segment, sentiment in segment_sentiments:
    print(f"Segment: {segment}\nSentiment: {sentiment}\n")

# Calculate overall sentiment score based on segment scores
overall_score = sum(sentiment['score'] for _, sentiment in segment_sentiments) / len(segment_sentiments)
overall_sentiment = {
    "label": "Overall Sentiment",
    "score": round(overall_score, 2)
}

print("Overall sentiment of the conversation:")
print(overall_sentiment)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Sentiment of the entire conversation:
{'label': 'Positive', 'score': 99.75}

Sentiment of each segment:
Segment: Customer Care 0:00:00
Sentiment: {'label': 'Negative', 'score': 63.0}

Segment: Hello, thank you for calling Healthcare Solutions. This is Jain Do speaking. How can I assist you today?
Sentiment: {'label': 'Positive', 'score': 97.56}

Segment: Client 0:00:08
Sentiment: {'label': 'Negative', 'score': 94.48}

Segment: Hi Jain, this is John Smith. I have a question about my recent bill from the New York branch.
Sentiment: {'label': 'Positive', 'score': 97.25}

Segment: Customer Care 0:00:13
Sentiment: {'label': 'Negative', 'score': 60.79}

Segment: Sure John, I can help with that. I see that you were charged $200 on June 20, 2023.
Sentiment: {'label': 'Positive', 'score': 74.0}

Segment: Client 0:00:20
Sentiment: {'label': 'Negative', 'score': 95.56}

Segment: Let me check if there are any available discounts or promotions for you.
Sentiment: {'label': 'Negative', 'score': 99.5